<a href="https://colab.research.google.com/github/ArturoSirvent/TFG_notebooks/blob/main/single_input/analisis_9_2_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt 
import os
import pandas as pd
import glob 
import re


In [ ]:
#ahora vamos a cargar los datos dt segun telescopio, luego las evaluaremos tanto por RUN como juntas
datos_dir="/content/drive/MyDrive/analisis_datos_tfg_inicial_hasta_4_02_21/ultimo_analisis_9_2_21"
os.chdir(datos_dir)
#vamos a hacer un bucle for para ir por los telescopios
for i in range(1,2):
  print(i)
  names_aux=glob.glob("gamma_tel_"+str(i)+"_*.dt")
  print(names_aux)
  for j in range(len(names_aux)):
    print(names_aux[j],end="\n")
    datos_aux=[]
    aux_df=pd.read_csv(os.path.join(datos_dir,names_aux[j]),sep='  ',names=["1","2","3","4","5","6"],engine="python")
    #ahora la procesamos y la guardamos en un npy
    value_auf=aux_df[['1','3','4','5']].copy()
    del aux_df
    #tenemos que agupar los valores y normalizarlos 
    value_auf.loc[value_auf["5"]<0]=0
    max_aux=np.amax(value_auf["5"])
    value_auf["5"]=value_auf["5"]/max_aux
    x_minimo=min(value_auf['3'])
    y_minimo=min(value_auf['4'])
    value_auf['3']=value_auf['3'].apply(lambda x: round((x-x_minimo)/333))
    value_auf['4']=value_auf['4'].apply(lambda x: round((x-y_minimo)/192))
    event_aux=value_auf["1"].unique()
    print(event_aux)

    for k in event_aux:
      #cada evento tiene que ponerse en una imagen con sus valores
      array_aux=value_auf.loc[value_auf["1"]==k][["3","4","5"]]
      #lo que vamos a hacer es qponer los valores en una matriz creada de antemano y guardar esa matrix
      matrix_aux=np.zeros((60-5,131-38))
      matrix_aux[array_aux["3"].to_numpy()-5,array_aux["4"].to_numpy()-39]=array_aux["5"].to_numpy()
      datos_aux.append(matrix_aux)
    name_npy=os.path.join(datos_dir,"telesc_"+str(i)+"_RUN_"+str(j+1)+".npy")
    np.save(name_npy,np.array(datos_aux))

1
['gamma_tel_1_run_01.dt']
gamma_tel_1_run_01.dt
[   100      0    200 ... 199700 199800 199900]


In [ ]:
#para todos
datos_dir="/content/drive/MyDrive/analisis_datos_tfg_inicial_hasta_4_02_21/ultimo_analisis_9_2_21"
datos_total=[]
for i in range(1,2):
  os.chdir(datos_dir)
  archivos=glob.glob("telesc_"+str(i)+"_*.npy")
  matriz_total=np.zeros((60-5,131-38))
  datos_total=[]
  for k in archivos:
    datos=np.load(os.path.join(datos_dir,k))
    datos_total.append(datos)
    matriz_total+=sum(datos)
  nombre_aux="telescopio_"+str(i)+"_todas_RUNS"
  nombre_aux_suma_img="suma_total_"+nombre_aux
  
  #np.save(os.path.join(stats_dir,nombre_aux_suma_np),matrix_aux)
  plt.figure(figsize=(16,16))
  plt.imshow(matriz_total,aspect="auto")
  plt.colorbar()
  plt.title(nombre_aux_suma_img,fontsize=14)
  
  plt.tight_layout()
  plt.savefig(os.path.join(datos_dir,nombre_aux)+".png")
  plt.close()

In [ ]:
#ahora queremos ver los resultados
datos_dir_npy="/content/drive/MyDrive/analisis_datos_tfg_inicial_hasta_4_02_21/ultimo_analisis_9_2_21"
os.chdir(datos_dir_npy)
archivos=glob.glob("telesc_"+str(1)+"_*.npy")
for i in range(len(archivos)):
  datos=np.load(os.path.join(datos_dir_npy,archivos[i]),allow_pickle=True)
  nombre_aux=archivos[i].replace(".npy","")
  nombre_aux_suma_img="suma_total_"+nombre_aux
  matrix_aux=sum(datos)
  #np.save(os.path.join(stats_dir,nombre_aux_suma_np),matrix_aux)
  plt.figure(figsize=(16,16))
  plt.subplot(3,3,2)
  plt.imshow(matrix_aux,aspect="auto")
  plt.title(nombre_aux_suma_img,fontsize=14)
  #lo hacemos tambien para las indicendias totales, aquellas que superan un humbral

  #ahora calculamos los percentiles
  perc=["perc75","perc85","perc99.9"]
  vals_perc=[np.percentile(datos,75),np.percentile(datos,85),np.percentile(datos,99.9)]
  for j in range(len(perc)):
    datos_aux=datos.copy()
    nombre_aux_perc_img="suma_"+perc[j]+"_"+nombre_aux
    datos_aux[datos<vals_perc[j]]=0
    matrix_aux=sum(datos_aux)
    plt.subplot(3,3,4+j)
    plt.imshow(matrix_aux,aspect="auto")
    plt.title(nombre_aux_perc_img,fontsize=14)
    #lo hacemos tambien para las indicendias totales, aquellas que superan el percentil, las ponemos 
    #a valor 1 independientemente, y luego sumamos
    datos_aux=datos.copy()
    datos_aux[datos>=vals_perc[j]]=1.0
    datos_aux[datos<vals_perc[j]]=0
    matrix_aux=sum(datos_aux)
    nombre_aux_perc_img="incidencia_"+perc[j]+"_"+nombre_aux
    plt.subplot(3,3,7+j)
    plt.imshow(matrix_aux,aspect="auto")
    plt.title(nombre_aux_perc_img,fontsize=14)
  plt.tight_layout()
  plt.savefig(os.path.join(datos_dir_npy,nombre_aux)+".png")
  plt.close()